In [4]:
import numpy 
import pandas as pd
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [11]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])

    return numpy.array(dataX), numpy.array(dataY)

In [12]:
filename=['credit_risk_small_data_0.02.csv', 'credit_data_sim.csv']

In [21]:
data = pd.read_csv(filename[0])
(n,p) = data.shape
data = data.iloc[:,1:p]
data

,Revolving Credit Percentage,Capital Reserves,Num Late 60,Debt Ratio,Monthly Income,Num Credit Lines,Num Late Past 90,Num Real Estate,Num Late 90,Num Employees
0,0.390912,53,0,0.241776,10000,9,0,1,0,2
1,0.037574,64,0,0.323588,10500,11,0,2,0,0
2,0.002058,59,0,0.458093,16500,12,0,2,0,2
3,1.023988,36,0,0.203108,2766,4,0,0,0,3
4,0.966603,61,1,0.350550,3000,4,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
2400,0.058012,35,0,0.348095,4041,8,0,2,0,3
2401,0.024283,91,2,0.118773,5800,7,0,1,0,0
2402,0.271156,37,1,0.343996,9127,17,0,1,0,1
2403,0.923689,35,0,1.890141,2129,9,0,2,0,0


In [22]:
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)
data

array([[7.53783361e-05, 4.32432432e-01, 0.00000000e+00, ...,
        5.26315789e-02, 0.00000000e+00, 2.85714286e-01],
       [7.24528191e-06, 5.81081081e-01, 0.00000000e+00, ...,
        1.05263158e-01, 0.00000000e+00, 0.00000000e+00],
       [3.96807752e-07, 5.13513514e-01, 0.00000000e+00, ...,
        1.05263158e-01, 0.00000000e+00, 2.85714286e-01],
       ...,
       [5.22861467e-05, 2.16216216e-01, 1.02040816e-02, ...,
        5.26315789e-02, 0.00000000e+00, 1.42857143e-01],
       [1.78112016e-04, 1.89189189e-01, 0.00000000e+00, ...,
        1.05263158e-01, 0.00000000e+00, 0.00000000e+00],
       [5.61429342e-05, 1.21621622e-01, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [23]:
# split into train and test sets
train_size = int(len(data) * 0.67)
test_size = len(data) - train_size
train, test = data[0:train_size,:], data[train_size:len(data),:]

In [24]:
train

array([[7.53783361e-05, 4.32432432e-01, 0.00000000e+00, ...,
        5.26315789e-02, 0.00000000e+00, 2.85714286e-01],
       [7.24528191e-06, 5.81081081e-01, 0.00000000e+00, ...,
        1.05263158e-01, 0.00000000e+00, 0.00000000e+00],
       [3.96807752e-07, 5.13513514e-01, 0.00000000e+00, ...,
        1.05263158e-01, 0.00000000e+00, 2.85714286e-01],
       ...,
       [5.00611948e-05, 2.02702703e-01, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.42857143e-01],
       [7.70921905e-07, 2.29729730e-01, 0.00000000e+00, ...,
        5.26315789e-02, 0.00000000e+00, 2.85714286e-01],
       [1.92144636e-04, 5.81081081e-01, 1.02040816e-02, ...,
        5.26315789e-02, 0.00000000e+00, 2.85714286e-01]])

In [25]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [26]:
trainX

array([[7.53783361e-05],
       [7.24528191e-06],
       [3.96807752e-07],
       ...,
       [1.39497070e-04],
       [8.27619938e-07],
       [5.00611948e-05]])

In [27]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [29]:
trainX.shape

(1609, 1, 1)

In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
1609/1609 - 3s - loss: 7.1803e-04
Epoch 2/100
1609/1609 - 1s - loss: 7.1766e-04
Epoch 3/100
1609/1609 - 1s - loss: 7.1962e-04
Epoch 4/100
1609/1609 - 1s - loss: 7.1842e-04
Epoch 5/100
1609/1609 - 1s - loss: 7.2010e-04
Epoch 6/100
1609/1609 - 1s - loss: 7.1837e-04
Epoch 7/100
1609/1609 - 1s - loss: 7.1898e-04
Epoch 8/100
1609/1609 - 1s - loss: 7.1377e-04
Epoch 9/100
1609/1609 - 1s - loss: 7.1776e-04
Epoch 10/100
1609/1609 - 1s - loss: 7.1795e-04
Epoch 11/100
1609/1609 - 1s - loss: 7.1832e-04
Epoch 12/100
1609/1609 - 1s - loss: 7.1600e-04
Epoch 13/100
1609/1609 - 1s - loss: 7.1755e-04
Epoch 14/100
1609/1609 - 1s - loss: 7.1874e-04
Epoch 15/100
1609/1609 - 1s - loss: 7.1755e-04
Epoch 16/100
1609/1609 - 1s - loss: 7.1958e-04
Epoch 17/100
1609/1609 - 1s - loss: 7.1914e-04
Epoch 18/100
1609/1609 - 1s - loss: 7.1798e-04
Epoch 19/100
1609/1609 - 1s - loss: 7.1757e-04
Epoch 20/100
1609/1609 - 2s - loss: 7.1669e-04
Epoch 21/100
1609/1609 - 1s - loss: 7.1906e-04
Epoch 22/100
1609/1609